# Yosys direct synthesis

The pyosys.RTLIL target provides direct output of synthesizable designs from the intermediate IRL description.

## Examples


In [1]:
from cyhdl import *

@block
def unit(clk : ClkSignal,
         en : Signal.Type(bool),
         a : Signal,
         b  :Signal,
         q : Signal.Output):
    
    @always(clk.posedge)
    def worker():
        if en:
            q.next = a | b
        else:
            q.next = a ^ b
    
    return instances()
    

### Deriving from the RTLIL target

It is good practise to derive from the RTLIL class in a central location. This class can then easily be adapted with a .finalize() method to run custom commands on the synthesized elements.

In [2]:
from myirl.targets.pyosys import RTLIL

class RTL(RTLIL):
    def __init__(self):
        super().__init__("custom")
    def finalize(self, top, objs):
        tname = top.name
        design = self._design
        # design.run("hierarchy -check ; opt")
        self.log("FINALIZE implementation `%s` of `%s`" % (tname, top.obj.name), annotation='info')
        return [objs]

### Synthesis

We synthesize an implementation of this unit by creating the corresponding pin signals. Note that there can be several implementations, if this unit is instanced in various places with differing signal configurations (sizes).

In [3]:
rtl = RTL()
N = 12

clk = ClkSignal()
u, v, w, = [ Signal(intbv()[N:]) for _ in range(3) ]
en = Signal(bool())

u = unit(clk, en, u, v, w)
objs = u.elab(rtl)
design = objs[0]

 Adding module with name `unit` 
 FINALIZE implementation `unit` of `unit` 


If the design is a flat structure with no hierarchy, we can simply display it using the `inline` specification.

Care must be taken on big designs, as they may create large files that the browser is unable to display.

In [4]:
design.display_rtl(inline = True)

### Conclusion

We can verify the logic being created for this unit as follows:
The `en` if..else construct creates a multiplexer primitive, selecting from the two prepended logic gates. More selection scenarios would create cascaded mulitplexers (this is left as an example).

Finally, the output value is fed into a data flip flop `$dff`.

The octagon pads represent the interface of the unit function. Our top level signals are not visible.

## Hierarchical design display

The `.display_rtl()` method can not deal with multi page output, i.e. only a module at a time ca be displayed. A hierarchical design requires to pass a `selection` parameter.

First, a top level module is created with internal signals connected to `unit` instances

In [5]:
@block
def top():
    N = 12

    clk = ClkSignal()
    u0, v0, w0, = [ Signal(intbv()[N:]) for _ in range(3) ]
    u1, v1, w1, = [ Signal(intbv()[N-2:]) for _ in range(3) ]
    en = Signal(bool())

    inst0 = unit(clk, en, u0, v0, w0)
    inst1 = unit(clk, en, u1, v1, w1)

    return instances()

In [6]:
t = top()
objs = t.elab(rtl)
d = objs[0]
d.display_rtl(selection = top, inline = True)

 Module top: Existing instance unit, rename to unit_1 
 Module top: Existing instance unit, rename to unit_2 
 Adding module with name `unit_2` 
 Adding module with name `unit_1` 
 Adding module with name `top` 
 FINALIZE implementation `top` of `top` 


We observe the internal signals being represented by wedges, whereas the unit instances are shown as black box modules..

# Synthesis for target

To synthesize for a specific target, a `@cyrite_factory.Module` class is first created. Then, it is instanced by passing an `Architecture` class, containing the configuration for the FPGA target and associated toolchain.

See also [Design](methods.ipynb).

The `@cyrite_factory.Module` object represents the top level design in various configurations. Typically, a user would supply a `.build()` method that calls the entire flow process with the result of a binary file ready to download to the target.

However since this is very system specific, the ways to do are intentially left open and can be freely implemented by the user.

## Customizing RTLIL targets

When the `.elab()` method is called, the design is elaborated as RTLIL and a list of design elements is returned, the first being a RTLIL Design handle.
The `.finalize()` method is called last inside elaboration, which can perform some optimizations or emissions to specific targets.



In [7]:
from myirl.targets import pyosys

class MyRTL(pyosys.RTLIL):
    def finalize(self, top, objs = None):
        tname = top.name
        design = self._design
        design.run("hierarchy -top %s" % tname)
        print(80 * '=')
        design.write_verilog(name = top.obj.name)
        design.run("flatten; ls; select %s; stat" % tname)
        
        return [design]

The `.finalize()` function must return the created elements in a list, which is in turn returned from the `.elab()` call.

In [8]:
rtl = MyRTL("top")
t = top()
objs = t.elab(rtl)
d = objs[0]
d.display_rtl(selection = top, inline = True)

 Module top: Existing instance top, rename to top_1 
 Module top: Existing instance unit, rename to unit_3 
 Module top: Existing instance unit, rename to unit_4 
 Adding module with name `unit_4` 
 Adding module with name `unit_3` 
 Adding module with name `top_1` 
